# Setup and Imports

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set
import warnings
from pandas.io import gbq
#Custom Python Module with functions specifically for this project
import ChicagoDataCleaningFunctions as cd

# Get the Data

In [8]:
chicago_data = """
                    SELECT unique_key, date, primary_type, location_description, 
                            arrest, domestic, community_area, year
                    FROM `gdac-327115.Chicago.chicago2`
                    WHERE year >= 2011
               """
chicago_data = gbq.read_gbq(chicago_data, project_id="gdac-327115")

In [9]:
#Read in an Excel file with a one to one mapping between Chicago community areas and districts
chicago_districts = pd.read_excel("ChicagoCommunityAreas.xlsx")

In [10]:
#Data type can't be joined on an int
chicago_districts.community_area = chicago_districts["community_area"].astype("string")
chicago_data.community_area = chicago_data["community_area"].astype("string")

In [11]:
#Outer join the two data sets
chicago = chicago_data.merge(chicago_districts, how="outer", left_on="community_area", right_on="community_area", )

In [12]:
#Drop the community area variable since we have a community name variable
chicago.drop("community_area", axis = 1, inplace = True)

# Split the Data in Training and Test Sets

In [13]:
chicago_train = chicago.loc[chicago["year"] != 2021]
chicago_test = chicago.loc[chicago["year"] == 2021]

# Clean the Training Data

In [14]:
#Check for missing values
chicago_train.isnull().sum()

unique_key                 0
date                       0
primary_type               0
location_description    7844
arrest                     0
domestic                   0
year                       0
district_name            218
community_name           218
dtype: int64

In [18]:
%%capture --no-stdout
cd.chicago_data_cleaner(chicago_train, verbose=True)

Cleaning Started...

Successfully Cleaned Primary Type
Successfully Imputed Location
Successfully Cleaned Location
Successfully Added Month Column
Successfully Added Hour Column
Successfully Cleaned Community

Data Set Successfully Cleaned!


In [19]:
#Check to make sure the values were cleaned up
chicago_train.isnull().sum()

unique_key              0
date                    0
primary_type            0
location_description    0
arrest                  0
domestic                0
year                    0
district_name           0
community_name          0
Month                   0
Hour                    0
dtype: int64

In [20]:
#Check the dimensions
chicago_train.shape

(2814636, 11)

In [21]:
#Check the first few rows
chicago_train.head()

,unique_key,date,primary_type,location_description,arrest,domestic,year,district_name,community_name,Month,Hour
0,10225218,2015-09-05 23:00:00+00:00,ROBBERY,CTA,False,False,2015,Far Southeast,RIVERDALE,9,23
1,10341077,2015-12-10 05:15:00+00:00,BURGLARY,RESIDENCE,False,False,2015,Far Southeast,RIVERDALE,12,5
2,11861007,2019-10-14 22:47:00+00:00,OTHER OFFENSE,STREET,True,False,2019,Far Southeast,RIVERDALE,10,22
3,10599541,2016-07-15 22:30:00+00:00,INTERFERENCE WITH PUBLIC OFFICER,PARKING,True,False,2016,Far Southeast,RIVERDALE,7,22
4,10707134,2016-10-04 17:00:00+00:00,DECEPTIVE PRACTICE,STREET,False,False,2016,Far Southeast,RIVERDALE,10,17
